In [ ]:
import cv2
import numpy as np

# Define file paths
template_path = r"C:\Users\walid\Downloads\template.jpg"
test_image_path = r"C:\Users\walid\Downloads\DSPI\bul_train\6f9d3ff8-669a-4388-ab2a-9e627ead54fe_aug3.png"

# Load images
template = cv2.imread(template_path, 0)  # Load as grayscale
test_image = cv2.imread(test_image_path, 0)  # Load as grayscale

# Check if images are loaded successfully
if template is None or test_image is None:
    print("Error: Images not found or cannot be read!")
    exit()

# Resize template for display (optional)
display_scale = 0.5  # Adjust this value to fit your screen
template_display = cv2.resize(template, None, fx=display_scale, fy=display_scale)

# Use SIFT for feature detection
sift = cv2.SIFT_create()

keypoints1, descriptors1 = sift.detectAndCompute(template, None)
keypoints2, descriptors2 = sift.detectAndCompute(test_image, None)

# Match features with FLANN-based matcher (better for SIFT)
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(descriptors1, descriptors2, k=2)

# Apply Lowe's Ratio Test to filter good matches
good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:  # Stricter threshold (lower ratio)
        good_matches.append(m)

print("Number of good matches:", len(good_matches))

# Check if we have enough matches
if len(good_matches) < 4:
    print("Not enough matches found. Try better-quality images.")
else:
    # Extract matched keypoints
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)  # Template keypoints
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)  # Test keypoints

    # Compute Homography using RANSAC (lower threshold for better matching)
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)  # Map test image to template image

    # Get dimensions of the template image
    h, w = template.shape

    # Warp the test image to align with the template
    aligned_image = cv2.warpPerspective(test_image, H, (w, h))  # Output size matches template

    # Resize aligned image for display (optional)
    aligned_display = cv2.resize(aligned_image, None, fx=display_scale, fy=display_scale)

    # Create resizable windows
    cv2.namedWindow("Template Image", cv2.WINDOW_NORMAL)
    cv2.namedWindow("Aligned Image", cv2.WINDOW_NORMAL)

    # Show the images
    cv2.imshow("Template Image", template_display)
    cv2.imshow("Aligned Image", aligned_display)
    cv2.waitKey(0)
    cv2.destroyAllWindows()